## 准备数据

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(32)
    
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds = test_ds.map(prepare_mnist_features_and_labels)
    test_ds = test_ds.take(20000).shuffle(20000).batch(20000)
    return ds, test_ds

def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

## 建立模型

In [8]:
class myModel:
    def __init__(self):
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[28*28, 100],
                                                         minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[100, 10],
                                                         minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
    def __call__(self, x):
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 定义loss以及train loop

In [9]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):

    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    grads = tape.gradient(loss, model.trainable_variables)
    # update to weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test_step(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

def train(epoch, model, optimizer, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y) in enumerate(ds):
        loss, accuracy = train_one_step(model, optimizer, x, y)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

    return loss, accuracy
def test(model, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y) in enumerate(ds):
        loss, accuracy = test_step(model, x, y)

        
    print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

    return loss, accuracy

# 训练

In [10]:
train_ds, test_ds = mnist_dataset()
for epoch in range(5):
    loss, accuracy = train(epoch, model, optimizer, train_ds)
loss, accuracy = test(model, test_ds)

epoch 0 : loss 2.2736764 ; accuracy 0.0625
epoch 0 : loss 0.34273767 ; accuracy 0.90625
epoch 1 : loss 0.3138375 ; accuracy 0.9375
epoch 1 : loss 0.32103106 ; accuracy 0.90625
epoch 2 : loss 0.25856233 ; accuracy 0.9375
epoch 2 : loss 0.2955742 ; accuracy 0.90625
epoch 3 : loss 0.22511926 ; accuracy 0.9375
epoch 3 : loss 0.2737816 ; accuracy 0.9375
epoch 4 : loss 0.19638324 ; accuracy 0.9375
epoch 4 : loss 0.24829423 ; accuracy 0.9375
test loss 0.15834562 ; accuracy 0.949
